<a href="https://colab.research.google.com/github/FayedFahad/GDrive-Uploading/blob/master/ColabTorrent_Resumable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install Torrent Client**

In [ ]:
!apt-get install transmission-cli
!sudo echo "net.core.wmem_max = 1048576" >> /etc/sysctl.conf
!sudo echo "net.core.rmem_max = 4194304" >> /etc/sysctl.conf

## **PATH Input & Torrent Name**

In [ ]:
from os import chdir
chdir("/content/drive/Shared drives/Gd13Receive1/Torrent/Torrent Files")
!transmission-cli -D -g "/content/drive/Shared drives/Gd13Receive1/Torrent/Config" -w "/content/drive/Shared drives/Gd13Receive1/Torrent" sb.torrent

transmission-cli 2.92 (14714)
[2020-09-30 18:38:32.189] Transmission 2.92 (14714) started
[2020-09-30 18:38:32.191] RPC Server: Adding address to whitelist: 127.0.0.1
[2020-09-30 18:38:32.191] UDP: Failed to set receive buffer: requested 4194304, got 425984
[2020-09-30 18:38:32.191] UDP: Please add the line "net.core.rmem_max = 4194304" to /etc/sysctl.conf
[2020-09-30 18:38:32.191] UDP: Failed to set send buffer: requested 1048576, got 425984
[2020-09-30 18:38:32.191] UDP: Please add the line "net.core.wmem_max = 1048576" to /etc/sysctl.conf
[2020-09-30 18:38:32.192] DHT: Reusing old id
[2020-09-30 18:38:32.192] DHT: Bootstrapping from 40 IPv4 nodes
[2020-09-30 18:38:32.521] Port Forwarding (NAT-PMP): initnatpmp succeeded (0)
[2020-09-30 18:38:32.521] Port Forwarding (NAT-PMP): sendpublicaddressrequest succeeded (2)
[2020-09-30 18:38:34.093] Saved "/content/drive/Shared drives/Gd13Receive1/Torrent/Config/torrents/Savannah Bond.f6e50157eb4151db.torrent"
Progress: 0.0%, dl from 0 of 0 pe

## **Magnet Link**

In [ ]:
!transmission-cli -D -g "/content/drive/Shared drives/Additional Shared Drive/config" -w "/content/drive/Shared drives/" MAGNET_LINK_HERE

## *NOTES*

>1] Path where Torrent file is located
<BR>
>2] Create a config folder & Path of that folder
<BR>
>3]Path Of where video files will be downloaded

For Magnet Link
>1]Config Path
<br>
>2]Where to download Path
<br>
>3]Magnet Link 

In [ ]:
!transmission-cli -D -g "/content/drive/Shared drives/Additional Shared Drive/config" -w "/content/drive/Shared drives/" MAGNET_LINK_HERE

## SIMPLE TORRNTING
Install libtorrent

In [ ]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

Add From Torrent File

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

 Add From Magnet Link

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Start Download

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
